In [1]:
pip install tweepy pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install numpy==1.21.0

   ---------------------------------------- 14.0/14.0 MB 4.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
Note: you may need to restart the kernel to use updated packages.


  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.6.0 requires daal==2021.4.0, which is not installed.
scipy 1.13.0 requires numpy<2.3,>=1.22.4, but you have numpy 1.21.0 which is incompatible.

[notice] A new release of pip is available: 24.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
pip install pandas==1.3.0

   ---------------------------------------- 10.2/10.2 MB 4.5 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.4.4
    Uninstalling pandas-1.4.4:
      Successfully uninstalled pandas-1.4.4
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
pip install --upgrade tweepy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
import tweepy
import pandas as pd

# Twitter API credentials
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAMngugEAAAAAHs84A%2FalMvMIqCt%2FV5NCZfZNvf0%3DYHxkzFJDoE2RnUhfyXHL4tPfsTRvi5hQZV07GvkO12S9duuiHb'

# Authenticate with the Twitter API v2
client = tweepy.Client(bearer_token=bearer_token)

# Define a function to extract tweets using Twitter API v2
def get_tweets_v2(keyword, count):
    tweets = []
    query = f'{keyword} lang:en -is:retweet'
    
    response = client.search_recent_tweets(query=query, tweet_fields=['text'], max_results=count)
    
    for tweet in response.data:
        tweets.append(tweet.text)
    
    return tweets

# Extract tweets
keyword = "Happy"
tweet_count = 100  # max_results is capped at 100 per request in API v2
tweets = get_tweets_v2(keyword, tweet_count)

# Create a DataFrame
df = pd.DataFrame(tweets, columns=['tweet'])
df.to_csv('tweets.csv', index=False)



Forbidden: 403 Forbidden
When authenticating requests to the Twitter API v2 endpoints, you must use keys and tokens from a Twitter developer App that is attached to a Project. You can create a project via the developer portal.